In [ ]:
#$env:BEA_USERID = "C7AB3527-B334-4396-B1C9-837FAF4FB814"
import os
os.environ["BEA_USERID"] = "C7AB3527-B334-4396-B1C9-837FAF4FB814"

In [ ]:
import os
import json
import math
import time
import logging
from pathlib import Path
from typing import List, Dict, Any, Optional
import os, json, math, time, logging
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import yfinance as yf
from scipy import stats
import altair as alt

In [ ]:
# bea_sector_analysis.py
"""
import os, json, math, time, logging
from pathlib import Path
from typing import Dict, Any, List
import requests
import pandas as pd
import numpy as np
import yfinance as yf
from scipy import stats
"""
# --- config ---
BEA_USERID = os.getenv("BEA_USERID")  # REQUIRED
OUTPUT_DIR = Path("dashboard_output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
HIST_YEARS = list(range(2010, 2017))  # 2010-2016
RISK_FREE = 0.0175

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")

# -----------------------------
# 1) BEA helper: get data
# -----------------------------
BEA_BASE = "https://apps.bea.gov/api/data"

def bea_get_data(dataset: str, tableid: str, year_from: int, year_to: int, freq="A") -> pd.DataFrame:
    """
    Uses BEA API to get Data for a table. Must know TableID (use GetParameterValues to discover).
    Returns DataFrame with Year, IndustryCode (or LineDescription), DataValue.
    """
    if not BEA_USERID:
        raise RuntimeError("Set BEA_USERID environment variable (see https://apps.bea.gov/api/signup/).")
    params = {
        "UserID": BEA_USERID,
        "method": "GetData",
        "DataSetName": dataset,     # e.g., "GDPbyIndustry"
        "TableID": tableid,         # e.g., "XX" — must be looked up
        "Frequency": freq,
        "Year": f"{year_from}-{year_to}",
        "ResultFormat": "JSON"
    }
    r = requests.get(BEA_BASE, params=params, timeout=30)
    r.raise_for_status()
    j = r.json()
    data = j["BEAAPI"]["Results"]["Data"]
    df = pd.DataFrame(data)
    # normalize columns
    if "DataValue" in df.columns:
        df["DataValue"] = pd.to_numeric(df["DataValue"], errors="coerce")
    if "TimePeriod" in df.columns:
        df["Year"] = df["TimePeriod"].astype(int)
    return df

# -----------------------------
# 2) Company financials & ROCE
# -----------------------------
def fetch_financials_yf(ticker: str, years: List[int]) -> pd.DataFrame:
    """
    Tries to fetch annual income and balance sheet from yfinance.
    yfinance returns yearly frames keyed by periods; we'll attempt to pull "Operating Income" (or EBIT) and
    assets/current liabilities to compute ROCE for available years.
    Returns DataFrame indexed by Year with columns EBIT, TotalAssets, CurrentLiabilities.
    """
    tk = yf.Ticker(ticker)
    # yfinance may expose .financials (quarterly) and .get_financials? We'll attempt .financials / .balance_sheet / .cashflow
    try:
        # annual stat: use 'financials' & 'balance_sheet' which are quarterly by default in yfinance,
        # but Ticker has attribute .get_financials? We'll use the .financials (columns = years)
        inc = tk.financials.transpose()  # columns: items like 'Total Revenue', index: periods
    except Exception:
        inc = pd.DataFrame()
    try:
        bal = tk.balance_sheet.transpose()
    except Exception:
        bal = pd.DataFrame()
    # Normalize year index (yfinance gives Timestamp index); parse year
    def normalize(df):
        if df is None or df.empty:
            return pd.DataFrame()
        if isinstance(df.index, pd.DatetimeIndex):
            df_out = df.copy()
            df_out.index = df_out.index.year
            return df_out
        else:
            return df
    inc = normalize(inc)
    bal = normalize(bal)

    rows = []
    for y in years:
        row = {"Year": y}
        if y in inc.index:
            # Operating Income or EBIT? Try 'Operating Income' then 'Ebit'
            ebit = None
            for k in ["Operating Income", "OperatingIncome", "Ebit", "Earnings Before Interest and Taxes", "OperatingIncomeLoss"]:
                if k in inc.columns:
                    ebit = inc.loc[y, k]
                    break
            # fallback to 'Net Income' (less ideal)
            if ebit is None and "Net Income" in inc.columns:
                ebit = inc.loc[y, "Net Income"]
            row["EBIT"] = float(ebit) if pd.notnull(ebit) else None
        else:
            row["EBIT"] = None

        if y in bal.index:
            ta = bal.loc[y].get("Total Assets") if "Total Assets" in bal.columns else None
            cl = bal.loc[y].get("Total Current Liabilities") if "Total Current Liabilities" in bal.columns else None
            # yfinance uses different labels; try common alternatives
            if ta is None:
                for alt in ["TotalAssets", "Assets"]:
                    if alt in bal.columns:
                        ta = bal.loc[y, alt]
                        break
            if cl is None:
                for alt in ["Total Current Liabilities", "CurrentLiabilities", "TotalCurrentLiabilities"]:
                    if alt in bal.columns:
                        cl = bal.loc[y, alt]
                        break
            row["TotalAssets"] = float(ta) if pd.notnull(ta) else None
            row["CurrentLiabilities"] = float(cl) if pd.notnull(cl) else None
        else:
            row["TotalAssets"] = None
            row["CurrentLiabilities"] = None
        rows.append(row)
    return pd.DataFrame(rows).set_index("Year")

def compute_roce_from_financials(fin_df: pd.DataFrame) -> pd.Series:
    """
    ROCE = EBIT / (TotalAssets - CurrentLiabilities)
    Returns series indexed by Year (as percent)
    """
    out = {}
    for year, r in fin_df.iterrows():
        ebit = r.get("EBIT")
        ta = r.get("TotalAssets")
        cl = r.get("CurrentLiabilities")
        if ebit is None or ta is None or cl is None:
            out[year] = None
            continue
        denom = ta - cl
        if denom == 0 or denom is None or denom <= 0:
            out[year] = None
            continue
        out[year] = float(ebit) / float(denom) * 100.0  # percent
    return pd.Series(out)

# -----------------------------
# 3) Big-3 statistical helpers
# -----------------------------
def cohens_d(a, b):
    a = np.array(a); b = np.array(b)
    a = a[~np.isnan(a)]; b = b[~np.isnan(b)]
    if len(a) < 2 or len(b) < 2:
        return None
    na, nb = len(a), len(b)
    sa = np.std(a, ddof=1); sb = np.std(b, ddof=1)
    pooled = math.sqrt(((na-1)*sa*sa + (nb-1)*sb*sb)/(na+nb-2)) if na+nb-2>0 else None
    if pooled and pooled>0:
        return float((a.mean()-b.mean())/pooled)
    return None

def bootstrap_ci_diff(a, b, iters=2000, ci=95):
    a = np.array(a); b = np.array(b)
    a = a[~np.isnan(a)]; b = b[~np.isnan(b)]
    if len(a)==0 or len(b)==0:
        return None, None, None
    rng = np.random.default_rng(123)
    diffs = []
    for _ in range(iters):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(np.mean(sa)-np.mean(sb))
    diffs = np.array(diffs)
    mean_diff = diffs.mean()
    low = np.percentile(diffs, (100-ci)/2)
    high = np.percentile(diffs, 100-(100-ci)/2)
    return float(mean_diff), float(low), float(high)

# -----------------------------
# 4) Example mapping of industries -> company tickers
# -----------------------------
INDUSTRY_COMPANIES = {
    "Steel": ["NUE", "X", "STLD"],   # Nucor (NUE), U.S. Steel (X), Steel Dynamics (STLD)
    "Semiconductors": ["INTC","QCOM","TXN"],
    "Pharmaceuticals": ["PFE","JNJ","MRK"],
    "Tobacco": ["MO", "PM", "BTI"],
    "Computer Software": ["MSFT", "ORCL", "CRM"],
    "Household and Personal Care Products": ["PG", "KMB", "CL"],
    "Semiconductors": ["INTC", "QCOM", "TXN"],
    "Pharmaceuticals": ["PFE", "JNJ", "MRK"],
    "Entertainment": ["DIS", "WBD", "PARA"],
    "Aerospace and Defense": ["BA", "LMT", "RTX"],
    "Beverages": ["KO", "STZ", "PEP"],
    "Chemicals (Specialty)": ["PPG", "LIN", "BAYRY"],
    "Food Processing": ["KHC", "GIS", "CAG"],
    "Medical Products": ["BDX", "SYK", "BSX"],
    "Engineering and Construction": ["FLR", "ACM", "J"],
    "Restaurants and Catering": ["MCD", "DRI", "SBUX"],
    "Office Equipment and Services": ["XRX", "NCR", "NTAP"],
    "Apparel": ["VFC", "HBI", "RL"],
    "Furniture and Home Furnishings": ["MHK", "MAS", "MLKN"],
    "Chemicals (General)": ["DOW", "DD", "HUN"],
    "Electronic Products": ["AAPL", "HON", "DELL"],
    "Packaging and Containers": ["WRK", "BALL", "CCK"],
    "Metals and Mining": ["AA", "FCX", "NEM"],
    "Publishing and Newspapers": ["NWSA", "RRD", "GCI"],
    "Railroads": ["UNP", "CSX", "NSC"],
    "Hospitals and Healthcare Services": ["UNH", "HCA", "THC"],
    "Paper and Forest Products": ["WY", "IP", "BCC"],
    "Steel": ["NUE", "X", "STLD"],
    "Investment and Asset Management": ["BLK", "SCHW", "BEN"],
    "Telecommunications": ["T", "VZ", "CMCSA"],
    "Agricultural Processing": ["ADM", "TSN", "CHSCP"],
    "Petroleum": ["XOM", "CVX", "VLO"],
    "Insurance": ["MET", "PRU", "TRV"],
    "Food Retailing": ["KR", "ACI", "PUSHX"],
    "Trucking and Logistics": ["XPO", "CHRW", "JBHT"],
    "Hotels and Casinos": ["MAR", "LVS", "MGM"],
    "Motor Vehicle Parts": ["GM", "F", "LEA"],
    "Electrical Power": ["EXC", "DUK", "PCG"],
    "Motor Vehicles": ["GM", "F", "PCAR"],
    "Airlines": ["AAL", "DAL", "UAL"],
}


    # ... add other sectors from your list

# -----------------------------
# 5) Orchestrator: fetch, compute ROCE, compare to BEA sector
# -----------------------------
def run_sector_roce_analysis(industry_companies: Dict[str, List[str]], years: List[int]=HIST_YEARS):
    summary_rows = []
    # 1) Get BEA sector data placeholder: user must provide TableID mapping. Here we expect you to set mapping
    # Example BEA mapping for 'Steel' must be discovered via BEA GetParameterValues.
    BEA_TABLE_MAP = {
        # "Steel": "YOUR_TABLE_ID_FOR_STEEL_VALUE_ADDED",  # << you will need to fill these
    }

    # for each industry:
    for industry, tickers in industry_companies.items():
        logging.info(f"Processing industry {industry} with tickers {tickers}")
        # 1) fetch BEA industry macro (user must fill TableID)
        bea_df = None
        tableid = BEA_TABLE_MAP.get(industry)
        if tableid:
            try:
                bea_df = bea_get_data("GDPbyIndustry", tableid, years[0], years[-1])
            except Exception as e:
                logging.warning(f"BEA fetch failed for {industry}: {e}")
        else:
            logging.info(f"No BEA TableID configured for {industry} — skipping BEA step.")

        # 2) compute company ROCE time series
        company_roces = {}
        for t in tickers:
            try:
                fin = fetch_financials_yf(t, years)
                roce = compute_roce_from_financials(fin)
                company_roces[t] = roce
            except Exception as e:
                logging.warning(f"Failed financials for {t}: {e}")
                company_roces[t] = pd.Series({y:np.nan for y in years})
        # 3) compute sector average ROCE across companies (simple mean, year by year)
        roce_df = pd.DataFrame(company_roces)
        sector_mean = roce_df.mean(axis=1, skipna=True)  # index = Year
        # 4) For each company, do Big-3 comparing company's ROCE vector to sector_mean vector
        for t in tickers:
            comp_series = roce_df[t]
            # align years
            comp_vals = comp_series.reindex(years).values.astype(float)
            sector_vals = sector_mean.reindex(years).values.astype(float)
            # remove nan pairs
            mask = ~np.isnan(comp_vals) & ~np.isnan(sector_vals)
            if mask.sum() < 2:
                p_val = None; d = None; ci_mean = (None,None,None)
            else:
                try:
                    # paired t-test
                    tstat, p_val = stats.ttest_rel(comp_vals[mask], sector_vals[mask])
                except Exception:
                    p_val = None
                d = cohens_d(comp_vals[mask], sector_vals[mask])
                ci_mean = bootstrap_ci_diff(comp_vals[mask], sector_vals[mask])
            # record
            summary_rows.append({
                "industry": industry,
                "ticker": t,
                "company_avg_roce": float(np.nanmean(comp_vals)) if np.nanmean(comp_vals)==np.nanmean(comp_vals) else None,
                "sector_avg_roce": float(np.nanmean(sector_vals)) if np.nanmean(sector_vals)==np.nanmean(sector_vals) else None,
                "p_value": p_val,
                "effect_size_d": d,
                "ci_mean_diff": ci_mean,
                "n_years": int(mask.sum())
            })
    df_summary = pd.DataFrame(summary_rows)
    # Save results
    df_summary.to_csv(OUTPUT_DIR / "sector_roce_summary.csv", index=False)
    df_summary.to_json(OUTPUT_DIR / "sector_roce_summary.json", orient="records")
    logging.info(f"Wrote {len(df_summary)} summary rows to {OUTPUT_DIR}")
    return df_summary

# -----------------------------
# 6) Example run
# -----------------------------
if __name__ == "__main__":
    # run for Steel + sample industries
    df_out = run_sector_roce_analysis({"Steel": ["NUE","X","STLD"], "Semiconductors":["INTC","QCOM","TXN"], "Pharmaceuticals":["PFE","JNJ","MRK"]})
    print(df_out.head(10))


/tmp/ipython-input-567894732.py:289: RuntimeWarning: Mean of empty slice
  "company_avg_roce": float(np.nanmean(comp_vals)) if np.nanmean(comp_vals)==np.nanmean(comp_vals) else None,
/tmp/ipython-input-567894732.py:290: RuntimeWarning: Mean of empty slice
  "sector_avg_roce": float(np.nanmean(sector_vals)) if np.nanmean(sector_vals)==np.nanmean(sector_vals) else None,
/tmp/ipython-input-567894732.py:289: RuntimeWarning: Mean of empty slice
  "company_avg_roce": float(np.nanmean(comp_vals)) if np.nanmean(comp_vals)==np.nanmean(comp_vals) else None,
/tmp/ipython-input-567894732.py:290: RuntimeWarning: Mean of empty slice
  "sector_avg_roce": float(np.nanmean(sector_vals)) if np.nanmean(sector_vals)==np.nanmean(sector_vals) else None,


          industry ticker company_avg_roce sector_avg_roce p_value  \
0            Steel    NUE             None            None    None   
1            Steel      X             None            None    None   
2            Steel   STLD             None            None    None   
3   Semiconductors   INTC             None            None    None   
4   Semiconductors   QCOM             None            None    None   
5   Semiconductors    TXN             None            None    None   
6  Pharmaceuticals    PFE             None            None    None   
7  Pharmaceuticals    JNJ             None            None    None   
8  Pharmaceuticals    MRK             None            None    None   

  effect_size_d        ci_mean_diff  n_years  
0          None  (None, None, None)        0  
1          None  (None, None, None)        0  
2          None  (None, None, None)        0  
3          None  (None, None, None)        0  
4          None  (None, None, None)        0  
5          None  

/tmp/ipython-input-567894732.py:289: RuntimeWarning: Mean of empty slice
  "company_avg_roce": float(np.nanmean(comp_vals)) if np.nanmean(comp_vals)==np.nanmean(comp_vals) else None,
/tmp/ipython-input-567894732.py:290: RuntimeWarning: Mean of empty slice
  "sector_avg_roce": float(np.nanmean(sector_vals)) if np.nanmean(sector_vals)==np.nanmean(sector_vals) else None,


In [ ]:
# Iterate through the INDUSTRY_COMPANIES dictionary and print the mapping
print("Industries and their corresponding companies:")
for industry, companies in INDUSTRY_COMPANIES.items():
    print(f"\nIndustry: {industry}")
    print("Companies:", ", ".join(companies))

Industries and their corresponding companies:

Industry: Steel
Companies: NUE, X, STLD

Industry: Semiconductors
Companies: INTC, QCOM, TXN

Industry: Pharmaceuticals
Companies: PFE, JNJ, MRK

Industry: Tobacco
Companies: MO, PM, BTI

Industry: Computer Software
Companies: MSFT, ORCL, CRM

Industry: Household and Personal Care Products
Companies: PG, KMB, CL

Industry: Entertainment
Companies: DIS, WBD, PARA

Industry: Aerospace and Defense
Companies: BA, LMT, RTX

Industry: Beverages
Companies: KO, STZ, PEP

Industry: Chemicals (Specialty)
Companies: PPG, LIN, BAYRY

Industry: Food Processing
Companies: KHC, GIS, CAG

Industry: Medical Products
Companies: BDX, SYK, BSX

Industry: Engineering and Construction
Companies: FLR, ACM, J

Industry: Restaurants and Catering
Companies: MCD, DRI, SBUX

Industry: Office Equipment and Services
Companies: XRX, NCR, NTAP

Industry: Apparel
Companies: VFC, HBI, RL

Industry: Furniture and Home Furnishings
Companies: MHK, MAS, MLKN

Industry: Chemica

In [ ]:
#!/usr/bin/env python3
"""
bea_sector_roce_workflow.py

- Uses BEA API to find relevant industry table IDs (by keyword) and pull annual series (2010-2016)
- Fetches company financials via yfinance and computes ROCE (EBIT / (TotalAssets - CurrentLiabilities))
- Compares company ROCE to sector ROCE (2010-2016): p-value (paired t), Cohen's d, bootstrap CI
- Exports CSV/JSON + Altair charts in dashboard_output/

Requirements:
  pip install yfinance pandas numpy scipy requests altair

Usage:
  - Set BEA_USERID env var (recommended). If not set, the script uses the default key embedded below.
  - Run: python bea_sector_roce_workflow.py
"""
import os
import json
import math
import time
import logging
from pathlib import Path
from typing import List, Dict, Any, Optional

import requests
import pandas as pd
import numpy as np
import yfinance as yf
from scipy import stats
import altair as alt

# ---------- Configuration ----------
# Prefer env variable; fallback to key you provided (rotate key as needed)
BEA_USERID = os.getenv("BEA_USERID", "C7AB3527-B334-4396-B1C9-837FAF4FB814")
BEA_BASE = "https://apps.bea.gov/api/data"
BEA_PARAM_BASE = "https://apps.bea.gov/api/parametervalues"

OUTPUT_DIR = Path("dashboard_output")
CHARTS_DIR = OUTPUT_DIR / "charts"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
CHARTS_DIR.mkdir(parents=True, exist_ok=True)

YEARS = list(range(2018, 2024))   # Updated to 2018-2023 inclusive
BOOTSTRAP_ITERS = 2000

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")

# ---------- Helper: BEA API ----------
def bea_get_parameter_values(param_name: str, datasetname: str = "GDPbyIndustry") -> List[Dict[str,str]]:
    """
    Return list of parameter values for a BEA parameter (useful to discover TableIDs or Industry codes).
    param_name examples: "TableID", "Industry"
    """
    params = {
        "UserID": BEA_USERID,
        "method": "GetParameterValues",
        "DatasetName": datasetname,
        "ParameterName": param_name,
        "ResultFormat": "JSON"
    }
    r = requests.get(BEA_PARAM_BASE, params=params, timeout=30)
    r.raise_for_status()
    j = r.json()
    return j.get("BEAAPI", {}).get("Results", {}).get("ParamValue", [])

def bea_get_data(dataset: str, tableid: str, year_from: int, year_to: int, frequency: str = "A") -> pd.DataFrame:
    """
    Fetch BEA data for a specific TableID and year range.
    Returns DataFrame with at least Year and DataValue (numeric).
    """
    params = {
        "UserID": BEA_USERID,
        "method": "GetData",
        "DataSetName": dataset,
        "TableID": tableid,
        "Frequency": frequency,
        "Year": f"{year_from}-{year_to}",
        "ResultFormat": "JSON"
    }
    r = requests.get(BEA_BASE, params=params, timeout=60)
    r.raise_for_status()
    j = r.json()
    data = j.get("BEAAPI", {}).get("Results", {}).get("Data", [])
    if not data:
        return pd.DataFrame()
    df = pd.DataFrame(data)
    # Normalize
    if "DataValue" in df.columns:
        df["DataValue"] = pd.to_numeric(df["DataValue"], errors="coerce")
    if "TimePeriod" in df.columns:
        df["Year"] = df["TimePeriod"].astype(int)
    return df

# ---------- Financials & ROCE ----------
def fetch_financials_yf(ticker: str, years: List[int]) -> pd.DataFrame:
    """
    Attempt to extract annual EBIT and balance-sheet items from yfinance.
    Returns DataFrame indexed by Year with columns ['EBIT','TotalAssets','CurrentLiabilities'] (may contain Nones).
    """
    tk = yf.Ticker(ticker)
    # yfinance returns DataFrames with DatetimeIndex for .financials and .balance_sheet
    out_rows = []
    try:
        inc = tk.financials.transpose()  # may be empty
        if inc.empty:
            logging.warning(f"yfinance: No income statement data for {ticker}")
    except Exception as e:
        logging.warning(f"yfinance: Error fetching income statement for {ticker}: {e}")
        inc = pd.DataFrame()

    try:
        bal = tk.balance_sheet.transpose()
        if bal.empty:
             logging.warning(f"yfinance: No balance sheet data for {ticker}")
    except Exception as e:
        logging.warning(f"yfinance: Error fetching balance sheet for {ticker}: {e}")
        bal = pd.DataFrame()

    # Normalize yearly index to ints if possible
    def to_year_index(df):
        if df is None or df.empty:
            return pd.DataFrame()
        if isinstance(df.index, pd.DatetimeIndex):
            df2 = df.copy()
            df2.index = df2.index.year
            return df2
        else:
            return df

    inc = to_year_index(inc)
    bal = to_year_index(bal)

    for y in years:
        row = {"Year": y, "EBIT": None, "TotalAssets": None, "CurrentLiabilities": None}
        if not inc.empty and y in inc.index:
            # Try common names
            for key in ["Operating Income", "OperatingIncome", "OperatingIncomeLoss", "Ebit", "Earnings before interest and taxes", "EarningsBeforeInterestAndTaxes"]:
                if key in inc.columns:
                    val = inc.loc[y, key]
                    row["EBIT"] = float(val) if pd.notnull(val) else None
                    break
            # fallback to Net Income
            if row["EBIT"] is None and "Net Income" in inc.columns:
                val = inc.loc[y, "Net Income"]
                row["EBIT"] = float(val) if pd.notnull(val) else None
        if not bal.empty and y in bal.index:
            # Total assets
            for key in ["Total Assets", "TotalAssets","Assets"]:
                if key in bal.columns:
                    val = bal.loc[y, key]
                    row["TotalAssets"] = float(val) if pd.notnull(val) else None
                    break
            for key in ["Total Current Liabilities", "Current Liabilities", "TotalCurrentLiabilities","CurrentLiabilities"]:
                if key in bal.columns:
                    val = bal.loc[y, key]
                    row["CurrentLiabilities"] = float(val) if pd.notnull(val) else None
                    break
        if row["EBIT"] is None or row["TotalAssets"] is None or row["CurrentLiabilities"] is None:
             logging.info(f"yfinance: Missing data for {ticker} in year {y} (EBIT: {row['EBIT']}, TA: {row['TotalAssets']}, CL: {row['CurrentLiabilities']})")
        out_rows.append(row)
    dfout = pd.DataFrame(out_rows).set_index("Year")
    return dfout

def compute_roce(fin_df: pd.DataFrame) -> pd.Series:
    """
    ROCE (%) = EBIT / (TotalAssets - CurrentLiabilities) * 100
    Returns pd.Series indexed by Year (floats or NaN)
    """
    out = {}
    for year, row in fin_df.iterrows():
        ebit = row.get("EBIT")
        ta = row.get("TotalAssets")
        cl = row.get("CurrentLiabilities")
        if ebit is None or ta is None or cl is None:
            out[int(year)] = np.nan
            continue
        denom = ta - cl
        if denom is None or denom == 0:
            out[int(year)] = np.nan
            continue
        out[int(year)] = (float(ebit) / float(denom)) * 100.0
    return pd.Series(out)

# ---------- Big-3 stats ----------
def cohens_d(a: np.ndarray, b: np.ndarray) -> Optional[float]:
    a = np.array(a); b = np.array(b)
    a = a[~np.isnan(a)]; b = b[~np.isnan(b)]
    if len(a) < 2 or len(b) < 2:
        return None
    na, nb = len(a), len(b)
    sa = a.std(ddof=1); sb = b.std(ddof=1)
    pooled = math.sqrt(((na-1)*sa*sa + (nb-1)*sb*sb) / (na+nb-2)) if na+nb-2>0 else None
    if pooled and pooled > 0:
        return float((a.mean() - b.mean()) / pooled)
    return None

def bootstrap_diff_ci(a: np.ndarray, b: np.ndarray, iters:int=BOOTSTRAP_ITERS, ci:float=95.0):
    a = np.array(a); b = np.array(b)
    a = a[~np.isnan(a)]; b = b[~np.isnan(b)]
    if len(a) < 2 or len(b) < 2:
        return None, None, None
    rng = np.random.default_rng(12345)
    diffs = []
    for _ in range(iters):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(np.mean(sa) - np.mean(sb))
    arr = np.array(diffs)
    low = np.percentile(arr, (100-ci)/2)
    high = np.percentile(arr, 100-(100-ci)/2)
    return float(arr.mean()), float(low), float(high)

# ---------- High-level orchestrator ----------
# Map industries to tickers (include Nucor, US Steel, Steel Dynamics)
INDUSTRY_COMPANIES = {
    "Steel": ["NUE", "X", "STLD"],   # Nucor (NUE), U.S. Steel (X), Steel Dynamics (STLD)
    "Tobacco": ["MO", "RAI", "PM"],  # Altria, Reynolds American, Philip Morris Intl.
    "Computer Software": ["MSFT", "ORCL", "CRM"],
    "Pharmaceuticals": ["PFE", "JNJ", "MRK"],
    # Add other industries/topics as needed...
}

def run_workflow(industry_companies: Dict[str, List[str]], years: List[int]=YEARS):
    results = []
    # Optionally: discover candidate TableIDs for GDPbyIndustry (user may later refine)
    # We'll attempt to find TableIDs whose descriptions contain the industry name (case-insensitive)
    logging.info("Fetching BEA TableIDs metadata (may be large)...")
    try:
        paramvals = bea_get_parameter_values("TableID")
        # paramvals is list of dicts with 'ParameterValue' and maybe 'Description'
    except Exception as e:
        logging.warning(f"Could not fetch TableID parameter list from BEA: {e}")
        paramvals = []

    # build a simple mapping candidate: find TableIDs where description or value contains industry keyword
    tableid_candidates = {}
    for industry in industry_companies.keys():
        candidates = []
        key = industry.lower()
        for p in paramvals:
            pv = str(p.get("ParameterValue",""))
            desc = str(p.get("Description","") or "")
            if key in pv.lower() or key in desc.lower():
                candidates.append({"TableID": pv, "Description": desc})
        tableid_candidates[industry] = candidates

    # Save candidate TableIDs for review
    with open(OUTPUT_DIR / "bea_tableid_candidates.json","w") as fh:
        json.dump(tableid_candidates, fh, indent=2)
    logging.info("Wrote BEA TableID candidate list to dashboard_output/bea_tableid_candidates.json")

    # NOTE: The candidate list is for human review. Automated mapping risk: false positives.
    # For demo, we attempt to use frequently useful table: "GDPbyIndustry" TableID "A... something".
    # However, because BEA table IDs vary, you should choose the appropriate TableID from the candidate file.
    # For this script, we will not auto-call an arbitrary TableID; instead we will proceed with company ROCE computation
    # and write a results file to be merged with BEA series once you pick exact TableIDs.

    # 1) compute ROCE for each company
    for industry, tickers in industry_companies.items():
        logging.info(f"Processing industry {industry} tickers: {tickers}")
        # compute company-level ROCE time series
        company_roce = {}
        for t in tickers:
            try:
                fin = fetch_financials_yf(t, years)
                roce_series = compute_roce(fin)
                company_roce[t] = roce_series.reindex(years)
                logging.info(f"Fetched ROCE for {t}: available years {list(company_roce[t].dropna().index)}")
            except Exception as e:
                logging.warning(f"Error computing ROCE for {t}: {e}")
                company_roce[t] = pd.Series({y:np.nan for y in years})

        # compute sector mean (simple mean across companies each year)
        roce_df = pd.DataFrame(company_roce)
        sector_mean = roce_df.mean(axis=1, skipna=True)

        # per-company big-3 vs sector mean
        for t in tickers:
            a = company_roce[t].values
            b = sector_mean.values
            # mask pairs
            valid = ~np.isnan(a) & ~np.isnan(b)
            n_pairs = int(np.sum(valid))
            if n_pairs >= 2:
                # paired t-test (year-over-year comparison)
                try:
                    tstat, p_val = stats.ttest_rel(a[valid], b[valid])
                except Exception:
                    tstat, p_val = None, None
                d = cohens_d(a[valid], b[valid])
                mean_diff, low_ci, high_ci = bootstrap_diff_ci(a[valid], b[valid])
            else:
                tstat, p_val, d, mean_diff, low_ci, high_ci = (None, None, None, None, None, None)

            results.append({
                "industry": industry,
                "ticker": t,
                "company_mean_roce": float(np.nanmean(a)) if not np.isnan(np.nanmean(a)) else None,
                "sector_mean_roce": float(np.nanmean(b)) if not np.isnan(np.nanmean(b)) else None,
                "n_years": n_pairs,
                "t_stat": tstat,
                "p_value": p_val,
                "cohens_d": d,
                "mean_diff": mean_diff,
                "ci_low": low_ci,
                "ci_high": high_ci
            })

        # produce a simple sector time-series chart (company lines + sector mean)
        ts_df = roce_df.reset_index().rename(columns={'index': 'Year'}).melt(id_vars="Year", var_name="Ticker", value_name="ROCE")
        # add sector mean series
        sector_mean_df = sector_mean.reset_index().rename(columns={0:"ROCE"})
        sector_mean_df["Ticker"] = f"{industry} (Sector Mean)"
        ts_all = pd.concat([ts_df, sector_mean_df], ignore_index=True, sort=False)
        try:
            chart = alt.Chart(ts_all).mark_line().encode(
                x=alt.X("Year:O"),
                y=alt.Y("ROCE:Q"),
                color=alt.Color("Ticker:N"),
                tooltip=["Year","Ticker","ROCE"]
            ).properties(width=800, height=350, title=f"{industry} ROCE (company lines + sector mean)")
            chart.save(CHARTS_DIR / f"{industry}_roce_timeseries.html")
            logging.info(f"Wrote chart for industry {industry}")
        except Exception as e:
            logging.warning(f"Could not write chart for {industry}: {e}")

    # save results
    dfres = pd.DataFrame(results)
    dfres.to_csv(OUTPUT_DIR / "sector_roce_summary.csv", index=False)
    dfres.to_json(OUTPUT_DIR / "sector_roce_summary.json", orient="records", double_precision=6)
    logging.info("Wrote sector_roce_summary.csv and .json to dashboard_output/")
    return dfres

if __name__ == "__main__":
    logging.info("Starting BEA + Sector ROCE workflow")
    df = run_workflow(INDUSTRY_COMPANIES)
    print(df.head(12))

/tmp/ipython-input-3156730679.py:298: RuntimeWarning: Mean of empty slice
  "company_mean_roce": float(np.nanmean(a)) if not np.isnan(np.nanmean(a)) else None,
/tmp/ipython-input-3156730679.py:298: RuntimeWarning: Mean of empty slice
  "company_mean_roce": float(np.nanmean(a)) if not np.isnan(np.nanmean(a)) else None,


             industry ticker  company_mean_roce  sector_mean_roce  n_years  \
0               Steel    NUE          34.226283         36.854123        3   
1               Steel      X                NaN         36.854123        0   
2               Steel   STLD          39.481963         36.854123        3   
3             Tobacco     MO          40.596899         41.288273        3   
4             Tobacco    RAI                NaN         41.288273        0   
5             Tobacco     PM          41.979647         41.288273        3   
6   Computer Software   MSFT          29.833842         15.494479        2   
7   Computer Software   ORCL          15.003071         15.494479        2   
8   Computer Software    CRM           1.646526         15.494479        2   
9     Pharmaceuticals    PFE          13.635835         14.755275        3   
10    Pharmaceuticals    JNJ          16.860865         14.755275        3   
11    Pharmaceuticals    MRK          13.769124         14.75527

1️⃣ BEA API: Dynamic Parameter Discovery

BEA provides an endpoint to list all TableIDs and parameters.

In [ ]:
import requests
import os

# Load your BEA API key securely
BEA_API_KEY = os.getenv("BEA_API_KEY")  # Set in environment variables or .env

def get_bea_parameters():
    """Return all available BEA parameter values for metadata discovery."""
    url = "https://apps.bea.gov/api/parametervalues"
    params = {
        "UserID": BEA_API_KEY,
        "method": "GetParameterValues",
        "methodname": "GetData",
        "ResultFormat": "JSON"
    }
    response = requests.get(url, params=params)
    data = response.json()

    # Extract list of parameters
    parameters = {}
    for item in data['BEAAPI']['Results']['Parameter']:
        name = item['ParameterName']
        values = [v['Key'] for v in item.get('Values', [])]
        parameters[name] = values
    return parameters

bea_params = get_bea_parameters()
print(bea_params.keys())  # List of all parameter names


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

What it does:

Queries BEA’s API for all available parameters and values.

Returns a dictionary: {'TableID': [...], 'Industry': [...], 'GeoFips': [...], 'Frequency': [...], ...}

Can be used to dynamically build queries without manually checking BEA documentation.

##2️⃣ Alpha Vantage: Metadata Discovery
*  Alpha Vantage dedicated access key is: 2M31LG9FZQGZX2PJ

While Alpha Vantage doesn’t have an explicit “list all symbols” endpoint for all exchanges, you can:

Use the sector performance endpoint:

In [ ]:
def get_alpha_vantage_sector(api_key):
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "SECTOR",
        "apikey": api_key
    }
    r = requests.get(url, params=params)
    return r.json()

sector_data = get_alpha_vantage_sector(os.getenv("ALPHA_VANTAGE_KEY"))
print(sector_data.keys())  # Gives all sector performance metrics


dict_keys([])


Use your symbol lists (download from exchanges or curated CSV):

NASDAQ, NYSE, AMEX

Map tickers to company names and sectors.

Use these tickers dynamically in queries like TIME_SERIES_DAILY_ADJUSTED or OVERVIEW.

##3️⃣ SEC EDGAR: Parameter Discovery

You can query CIK numbers, filings types (10-K, 10-Q, 8-K) via:

In [ ]:
#https://data.sec.gov/submissions/CIK########.json


##4️⃣ Implementation Plan in Your Dashboard

#1. Metadata Module:

* Queries BEA, Alpha Vantage, EDGAR for available parameters.
* Stores in cache (JSON or DB table) for fast lookup.
* Provides dropdowns/filters in dashboard frontend to select TableIDs, tickers, or filings.

#2. Dynamic Query Builder:

* User selects parameters (or default top-performing sectors).
* The backend constructs API queries automatically.
* ETL pulls, transforms, and stores the results.

#3. Interactive Dashboard:

* Let users explore all available datasets without needing API documentation.
* Show metadata summaries, e.g., “100+ TableIDs available for BEA GDP by Industry”.

##Dynamic SEC EDGAR crawler that automatically:

* Identifies companies from your investment list (tickers or names).

* Maps tickers/names to CIK numbers automatically.

* Retrieves all available filing types (10-K, 10-Q, 8-K, etc.) for each company.

* Stores results in a structured format for your ETL/EDA pipeline.

Here’s a Python implementation outline:

##1️⃣ Setup

In [ ]:
import requests
import pandas as pd
import time

# Base URL for SEC EDGAR
EDGAR_CIK_BASE = "https://www.sec.gov/files/company_tickers.json"  # Up-to-date ticker→CIK mapping
EDGAR_SUBMISSIONS = "https://data.sec.gov/submissions/CIK{}.json"  # Replace {} with 10-digit CIK
HEADERS = {"User-Agent": "WhitneyMoss-DataAnalyticsApp"}

# Example: List of investments to research
investments = ["AAPL", "MSFT", "NUE", "XOM"]  # Apple, Microsoft, Nucor, ExxonMobil


##2️⃣ Fetch CIK Numbers Automatically

In [ ]:
def get_cik_mapping():
    """Download SEC ticker-to-CIK mapping dynamically."""
    r = requests.get(EDGAR_CIK_BASE, headers=HEADERS)
    r.raise_for_status()
    data = r.json()
    time.sleep(1) # Add a small delay after fetching the mapping
    df = pd.DataFrame(data).T  # convert JSON to DataFrame
    df['cik_str'] = df['cik_str'].apply(lambda x: str(x).zfill(10))  # pad to 10 digits
    return df[['ticker', 'cik_str', 'title']]

cik_df = get_cik_mapping()

HTTPError: 403 Client Error: Forbidden for url: https://www.sec.gov/files/company_tickers.json

In [ ]:
# Retry fetching company tickers from SEC
cik_df = get_cik_mapping()

HTTPError: 403 Client Error: Forbidden for url: https://www.sec.gov/files/company_tickers.json

##3️⃣ Retrieve Filings for Each Company

In [ ]:
def get_company_filings(ticker, cik_df):
    """Return all available filing types for a given company ticker."""
    try:
        cik = cik_df.loc[cik_df['ticker'] == ticker.upper(), 'cik_str'].values[0]
    except IndexError:
        print(f"Ticker {ticker} not found in SEC mapping.")
        return None

    url = EDGAR_SUBMISSIONS.format(cik)
    r = requests.get(url, headers=HEADERS)
    r.raise_for_status()
    data = r.json()

    # Extract filings
    filings = pd.DataFrame(data['filings']['recent'])
    filings['ticker'] = ticker.upper()
    return filings[['ticker', 'form', 'filingDate', 'accessionNumber']]

# Example usage:
all_filings = []
for ticker in investments:
    df = get_company_filings(ticker, cik_df)
    if df is not None:
        all_filings.append(df)
    time.sleep(0.5)  # SEC rate limit
all_filings_df = pd.concat(all_filings, ignore_index=True)


##4️⃣ Output / Storage

In [ ]:
# Save to CSV for ETL or dashboard integration
all_filings_df.to_csv("sec_filings_dynamic.csv", index=False)
print(all_filings_df.head())
